In [1]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
# Allows plots to appear directly in the notebook.
%matplotlib inline
# For dealing with some Accented characters (in Irish Place names)
import unidecode
# Date/time functionality
import datetime
import time
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
# look at some z-scores for inspecting outliers.
from scipy import stats
import seaborn as sb
# lookup lat/long and convert lat/long to national grid references.
import geopy
import pyproj

from patsy import dmatrices
from sklearn import metrics

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
#from sklearn.tree import export_text

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

import graphviz
from graphviz import Source
import dask.dataframe as dd
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

/home/team8/notebook/jupyterenv/local/lib/python2.7/site-packages/pyspark/context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [2]:
spark_df_leavetimes = spark.read.option("header","True") \
    .option("delimiter",";") \
    .option("inferSchema","true")\
    .csv("/home/team8/tmp/data/rt_leavetimes_DB_2018.txt")

In [3]:
spark_df_leavetimes.show(4, False)

+----------+------------------+-------+-----------+-----------+---------------+---------------+--------------+--------------+---------+----------+------------+-------------+--------+----------+---------------+------------------+----+
|DATASOURCE|DAYOFSERVICE      |TRIPID |PROGRNUMBER|STOPPOINTID|PLANNEDTIME_ARR|PLANNEDTIME_DEP|ACTUALTIME_ARR|ACTUALTIME_DEP|VEHICLEID|PASSENGERS|PASSENGERSIN|PASSENGERSOUT|DISTANCE|SUPPRESSED|JUSTIFICATIONID|LASTUPDATE        |NOTE|
+----------+------------------+-------+-----------+-----------+---------------+---------------+--------------+--------------+---------+----------+------------+-------------+--------+----------+---------------+------------------+----+
|DB        |01-JAN-18 00:00:00|5972116|12         |119        |48030          |48030          |48012         |48012         |2693211  |null      |null        |null         |null    |null      |null           |08-JAN-18 17:21:10|null|
|DB        |01-JAN-18 00:00:00|5966674|12         |119        |5

In [4]:
from pyspark.sql.functions import col, countDistinct

df_leavetimes_unique=spark_df_leavetimes.agg(*(countDistinct(col(c)).alias(c) for c in spark_df_leavetimes.columns))

In [7]:
df_leavetimes_unique.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46061)
Traceback (most recent call last):
  File "/home/team8/notebook/jupyterenv/local/lib/python2.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46061)
Traceback (most recent call last):
  File "/home/team8/notebook/jupyterenv/local/lib/python2.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46061)
Traceback

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46061)
Traceback (most recent call last):
  File "/home/team8/notebook/jupyterenv/local/lib/python2.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46061)

In [6]:
cols=('DATASOURCE','PASSENGERS','PASSENGERSIN','PASSENGERSOUT','DISTANCE','NOTE')
spark_df_leavetimes= spark_df_leavetimes.drop(*cols)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46061)
Traceback (most recent call last):
  File "/home/team8/notebook/jupyterenv/local/lib/python2.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46061)

In [ ]:
spark_df_leavetimes.show(4)

Reading data in to a dask dataframe

In [ ]:
dask_df_leavetimes=dd.read_csv('/home/team8/tmp/data/rt_leavetimes_DB_2018.txt',blocksize=25e6,sep=';' )